In [1]:
%pip -q install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
from langsmith import Client as LangSmithClient

DATASET_NAME='Config mgmt Rap Battle Dataset'
client = LangSmithClient()
datasets=client.list_datasets()
for dataset in datasets:
    if (dataset.name) == DATASET_NAME:
        client.delete_dataset(dataset_name=DATASET_NAME)
        print(dataset)

name='Config mgmt Rap Battle Dataset' description='Rap battle prompts.' data_type=<DataType.kv: 'kv'> id=UUID('93992fe3-1e24-4ae0-a495-0f6ab8b84885') created_at=datetime.datetime(2024, 2, 15, 10, 14, 37, 434561, tzinfo=datetime.timezone.utc) modified_at=datetime.datetime(2024, 2, 15, 10, 14, 37, 434561, tzinfo=datetime.timezone.utc) example_count=2 session_count=2 last_session_start_time=datetime.datetime(2024, 2, 15, 10, 30, 29, 86929)


In [4]:

# Inputs are provided to your model, so it know what to generate
dataset_inputs = [
    "a rap battle between Chef and Puppet",
    "a rap battle between Ansible and Pulumi",
    # ... add more as desired
]


In [5]:


# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=DATASET_NAME,
    description="Rap battle prompts.",
)
client.create_examples(
    inputs=[{"question": q} for q in dataset_inputs],
#    outputs=dataset_outputs,
    dataset_id=dataset.id,
)


In [6]:
from langchain import prompts
from langchain.schema import output_parser

# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant."), ("human", "{question}")]
)

from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

chain = prompt | chat | output_parser.StrOutputParser()
answer=chain.invoke({"question":"A rap battle between Kris and Toshaan"})
print(answer)

Alright, let's get this rap battle started
Between Kris and Toshaan, two artists so talented
They're about to go head to head, no holding back
So get ready for some sick rhymes and dope tracks

Kris, you're up first, show 'em what you got
Drop those lyrics like it's hot, make 'em all shocked
Your flow is smooth, your words so precise
You're the king of the mic, no need to think twice

Toshaan, it's your turn to shine on this stage
Bring the fire, unleash your rage
Your style is unique, your delivery so raw
You're the master of rhythm, the rap game's outlaw

Kris, you better step up your game, don't back down
Toshaan's coming at you, ready to take the crown
Your rhymes gotta hit hard, make 'em feel the impact
Show 'em why you're the best, leave 'em all in a trap

Toshaan, you're killing it, your flow so smooth
Your lyrics are lethal, they're making 'em move
You're the master of wordplay, the king of the beat
Kris, you better watch out, 'cause Toshaan's taking the heat

This rap battle i

In [7]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from langsmith.schemas import Example, Run

@run_evaluator
def is_empty(run: Run, example: Example | None = None):
    model_outputs = run.outputs["output"]
    score = not model_outputs.strip()
    return EvaluationResult(key="is_empty", score=score)


from langchain.smith import RunEvalConfig, run_on_dataset

# Define the evaluators to apply
verifier_llm=chat
eval_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria(
            {"creativity": "Is this submission creative, imaginative, or novel?"}
        ),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
        # "cot_qa",
        #        smith.RunEvalConfig.LabeledCriteria("conciseness"),
        #        smith.RunEvalConfig.LabeledCriteria("relevance"),
        #        smith.RunEvalConfig.LabeledCriteria("coherence"),
        #        smith.RunEvalConfig.LabeledCriteria("harmfulness"),
        #        smith.RunEvalConfig.LabeledCriteria("insensitivity"),
        #        smith.RunEvalConfig.LabeledCriteria("criminality"),
        #        smith.RunEvalConfig.LabeledCriteria("misogyny"),
        #        smith.RunEvalConfig.LabeledCriteria("controversiality"),
        #        smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        #        smith.RunEvalConfig.LabeledCriteria("maliciousness"),
    ],
    custom_evaluators=[is_empty],
    eval_llm=verifier_llm,
)

chain_results = client.run_on_dataset(
    dataset_name=DATASET_NAME,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    concurrency_level=5,
    verbose=True,
)


View the evaluation results for project 'artistic-fact-4' at:
https://smith.langchain.com/o/27713559-0b06-4e07-89cf-5339d432c5c3/datasets/99103d2b-cbe2-4e0a-acf9-09ad3b631b71/compare?selectedSessions=9867ea20-c41f-4344-9bf1-3810f403f7f8

View all tests for Dataset Config mgmt Rap Battle Dataset at:
https://smith.langchain.com/o/27713559-0b06-4e07-89cf-5339d432c5c3/datasets/99103d2b-cbe2-4e0a-acf9-09ad3b631b71
[------------------------------------------------->] 2/2

In [8]:
from pprint import pprint 

pprint(chain_results)

{'project_name': 'artistic-fact-4',
 'results': {'374514ac-2630-4d04-a7ad-c24d615ad864': {'execution_time': 7.429585,
                                                      'feedback': [EvaluationResult(key='creativity', score=1, value='Y', comment='To determine if the submission meets the criteria of creativity, we need to assess if it is creative, imaginative, or novel. \n\n1. The submission uses rhymes and wordplay to create a rap battle between Chef and Puppet. This demonstrates creativity in using a rap battle as a format for the submission.\n2. The submission introduces the characters of Chef and Puppet and creates a storyline for the rap battle. This shows imagination in developing a narrative within the rap battle context.\n3. The submission includes metaphors and comparisons to enhance the lyrical content. This displays creativity in using figurative language to make the rap battle more engaging.\n4. The submission maintains a consistent rhythm and flow throughout the verses. T